In [ ]:
!pip install --upgrade transformers pandas --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.2/91.2 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 57.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 58.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires pandas==2.2.2, but you have pandas 2.3.1 which is incompatible.
dask-cudf-cu12 25.6.0 requires pandas<2.2.4dev0,>=2.0, but you have pandas 2.3.1 which is incompatible.
cudf-cu12 25.6.0 requires pandas<2.2.4dev0,>=2.0, but you have pandas 2.3.1 which is incompatible.


In [ ]:
import transformers
print(transformers.__version__)

4.53.3


In [ ]:
import pandas as pd

# ✅ Download synthetic dataset (already created for you)


train_df = pd.read_csv("support_tickets_train.csv")
test_df = pd.read_csv("support_tickets_test.csv")

print("Train Samples:", train_df.shape, " Test Samples:", test_df.shape)
print(train_df.head(30))

Train Samples: (400, 2)  Test Samples: (100, 2)
                                            text  label
0   The printer is not responding to my computer      1
1          Introduce a monthly subscription plan      4
2       Why hasn’t my refund been processed yet?      3
3    I want fingerprint login in the next update      4
4       Why hasn’t my refund been processed yet?      3
5      Please change my registered email address      2
6                Please add more payment methods      4
7             I was charged twice, need a refund      3
8                 I was charged extra on my bill      0
9             I was charged twice, need a refund      3
10  I canceled the order but didn’t get a refund      3
11                   I need to reset my password      2
12             Why is the website so slow today?      1
13   I want fingerprint login in the next update      4
14   I want to deactivate my account temporarily      2
15           Can you add multi-language support?      4


In [ ]:
from google.colab import files
uploaded =files.upload()

Saving support_tickets_test.csv to support_tickets_test (1).csv
Saving support_tickets_train.csv to support_tickets_train (1).csv


In [ ]:
from transformers import AutoTokenizer
import torch

model_name = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)

def encode_data(texts, labels):
    encodings = tokenizer(
        texts.tolist(),
        padding=True,
        truncation=True,
        return_tensors="pt"
    )
    encodings["labels"] = torch.tensor(labels.tolist(), dtype=torch.long)
    return encodings

train_encodings = encode_data(train_df["text"], train_df["label"])
test_encodings = encode_data(test_df["text"], test_df["label"])

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [ ]:
from torch.utils.data import TensorDataset

train_dataset = TensorDataset(
    train_encodings["input_ids"],
    train_encodings["attention_mask"],
    train_encodings["labels"]
)

test_dataset = TensorDataset(
    test_encodings["input_ids"],
    test_encodings["attention_mask"],
    test_encodings["labels"]
)

In [ ]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer
from transformers.data.data_collator import DataCollatorWithPadding
import torch

model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=5)

training_args = TrainingArguments(
    run_name="./results",
    num_train_epochs=2,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    logging_dir="./logs",
    save_strategy="epoch",
    do_eval=True,
    output_dir="./results", # Added output_dir to specify where to save checkpoints and logs
)

# Define a custom data collator to handle the TensorDataset output
class CustomDataCollator:
    def __init__(self, tokenizer):
        self.tokenizer = tokenizer

    def __call__(self, features):
        input_ids = torch.stack([f[0] for f in features])
        attention_mask = torch.stack([f[1] for f in features])
        labels = torch.stack([f[2] for f in features])
        return {
            'input_ids': input_ids,
            'attention_mask': attention_mask,
            'labels': labels
        }

custom_collator = CustomDataCollator(tokenizer)


trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    data_collator=custom_collator # Use the custom data collator
)

trainer.train()

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Step,Training Loss


TrainOutput(global_step=100, training_loss=0.43473918914794923, metrics={'train_runtime': 178.7817, 'train_samples_per_second': 4.475, 'train_steps_per_second': 0.559, 'total_flos': 2897879376000.0, 'train_loss': 0.43473918914794923, 'epoch': 2.0})

In [ ]:
eval_results = trainer.evaluate()
print("✅ Evaluation Results:", eval_results)

✅ Evaluation Results: {'eval_loss': 0.0329977348446846, 'eval_runtime': 6.8327, 'eval_samples_per_second': 14.636, 'eval_steps_per_second': 1.903, 'epoch': 2.0}


In [ ]:
trainer.save_model("./fine_tuned_model")

In [ ]:
from torch.nn.functional import softmax

def predict_top3(text):
    inputs = tokenizer(text, return_tensors="pt")
    outputs = model(**inputs)
    probs = softmax(outputs.logits, dim=1)
    top3 = torch.topk(probs, k=3)
    return list(zip(top3.indices[0].tolist(), top3.values[0].tolist()))

print("✅ Prediction:", predict_top3("I need a refund because I was charged twice."))

✅ Prediction: [(3, 0.966978132724762), (0, 0.01455108541995287), (2, 0.00717901811003685)]
